In [3]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import pickle
import sys


In [4]:
# Making a GET request
r = requests.get('https://asia-en.onepiece-cardgame.com/cardlist/?series=556901')

# check status code for response received
# success code - 200
print(r)

<Response [200]>



value="569201" >EXTRA BOOSTER &lt;br class=&quot;spInline&quot;&gt;-MEMORIAL COLLECTION- [EB-01]</option>
value="569108" >BOOSTER PACK &lt;br class=&quot;spInline&quot;&gt;-TWO LEGENDS- [OP-08]</option>
value="569107" >BOOSTER PACK &lt;br class=&quot;spInline&quot;&gt;-500 YEARS IN THE FUTURE- [OP-07]</option>
value="569106" >BOOSTER PACK &lt;br class=&quot;spInline&quot;&gt;-WINGS OF THE CAPTAIN- [OP-06]</option>
value="569105" >BOOSTER PACK &lt;br class=&quot;spInline&quot;&gt;-AWAKENING OF THE NEW ERA- [OP-05]</option>
value="569104" >BOOSTER PACK &lt;br class=&quot;spInline&quot;&gt;-KINGDOMS OF INTRIGUE- [OP-04]</option>
value="569103" >BOOSTER PACK &lt;br class=&quot;spInline&quot;&gt;-PILLARS OF STRENGTH- [OP-03]</option>
value="569102" >BOOSTER PACK &lt;br class=&quot;spInline&quot;&gt;-PARAMOUNT WAR- [OP-02]</option>
value="569101" >BOOSTER PACK &lt;br class=&quot;spInline&quot;&gt;-ROMANCE DAWN- [OP-01]</option>
value="569014" >STARTER DECK &lt;br class=&quot;spInline&quot;&gt;-3D2Y- [ST-14]</option>
value="569013" >ULTRA DECK &lt;br class=&quot;spInline&quot;&gt;-The Three Brothers- [ST-13]</option>
value="569012" selected>STARTER DECK &lt;br class=&quot;spInline&quot;&gt;-Zoro and Sanji- [ST-12]</option>
value="569011" >STARTER DECK &lt;br class=&quot;spInline&quot;&gt;-Uta- [ST-11]</option>
value="569010" >ULTRA DECK &lt;br class=&quot;spInline&quot;&gt;-The Three Captains- [ST-10]</option>
value="569009" >STARTER DECK &lt;br class=&quot;spInline&quot;&gt;-Yamato- [ST-09]</option>
value="569008" >STARTER DECK &lt;br class=&quot;spInline&quot;&gt;-Monkey D. Luffy- [ST-08]</option>
value="569007" >STARTER DECK &lt;br class=&quot;spInline&quot;&gt;-Big Mom Pirates- [ST-07]</option>
value="569006" >STARTER DECK &lt;br class=&quot;spInline&quot;&gt;-Absolute Justice- [ST-06]</option>
value="569005" >STARTER DECK &lt;br class=&quot;spInline&quot;&gt;-ONE PIECE FILM edition- [ST-05]</option>
value="569004" >STARTER DECK &lt;br class=&quot;spInline&quot;&gt;-Animal Kingdom Pirates- [ST-04]</option>
value="569003" >STARTER DECK &lt;br class=&quot;spInline&quot;&gt;-The Seven Warlords of the Sea- [ST-03]</option>
value="569002" >STARTER DECK &lt;br class=&quot;spInline&quot;&gt;-Worst Generation- [ST-02]</option>
value="569001" >STARTER DECK &lt;br class=&quot;spInline&quot;&gt;-Straw Hat Crew- [ST-01]</option>
value="569901" >Promotion card</option>
value="569801" >Other Product Card</option>


In [5]:
soup = BeautifulSoup(r.content, 'html.parser')

site_card_data = soup.find_all(class_='modalCol')

In [6]:
card_data = []

for card in site_card_data:
    card_dict = {}

    card_dict['id'] = card.get('id')
    card_dict['card_number'] = card.get('id').split('-')[1].split('_')[0]
    card_dict['name'] = card.find('div', class_ = 'cardName').get_text()
    card_dict['category'] = card.find('div', class_ = 'infoCol').get_text().split('|')[2].strip()
    card_dict['cost'] = card.find('div', class_ = 'cost').get_text().strip('Cost').strip('Life')
    card_dict['attribute'] = card.find('div', class_='attribute').get_text().strip().split('\n')[1]
    card_dict['counter'] = card.find('div', class_='counter').get_text().strip('Counter')
    card_dict['color'] = card.find('div', class_='color').get_text().strip('Color')
    card_dict['feature'] = card.find('div', class_='feature').get_text()[4:]
    card_dict['effect'] = card.find('div', class_='text').get_text()[6:]
    card_dict['card_set'] = card.find('div', class_='getInfo').get_text().strip('Card Set(s)')
    card_dict['set_code'] = card.get('id').split('-')[0].split('_')[0]
    card_dict['image_src'] = 'https://asia-en.onepiece-cardgame.com/' + card.find('img', class_='lazy').get('data-src').strip('..')

    card_data.append(card_dict)

print(card_data[149])
print(card_data[27])

{'id': 'P-069', 'card_number': '069', 'name': 'Koala', 'catergory': 'CHARACTER', 'cost': '1', 'attribute': 'Strike', 'counter': '1000', 'color': 'Red', 'feature': 'Revolutionary Army', 'effect': '[Activate: Main] [Once Per Turn] Give up to 1 rested DON!! card to your Leader or 1 of your Characters.', 'card_set': 'Prize for May 2024 Meet-up even', 'set_code': 'P', 'image_src': 'https://asia-en.onepiece-cardgame.com//images/cardlist/card/P-069.png?240831'}
{'id': 'OP02-098_p1', 'card_number': '098', 'name': 'Koby (Parallel)', 'catergory': 'CHARACTER', 'cost': '3', 'attribute': 'Strike', 'counter': '1000', 'color': 'Black', 'feature': 'Navy', 'effect': "[On Play] You may trash 1 card from your hand: K.O. up to 1 of your opponent's Characters with a cost of 3 or less.", 'card_set': 'Promotion Pack Vol.3', 'set_code': 'OP02', 'image_src': 'https://asia-en.onepiece-cardgame.com//images/cardlist/card/OP02-098_p1.png?240831'}


In [127]:

conn = sqlite3.connect('OPTCGDB.db')
cur = conn.cursor()

cur.execute('''
    CREATE TABLE IF NOT EXISTS card_data (
        id TEXT PRIMARY KEY,
        card_number TEXT,
        name TEXT,
        cost TEXT,
        attribute TEXT,
        counter TEXT,
        color TEXT,
        feature TEXT,
        effect TEXT,
        card_set TEXT,
        set_code TEXT,
        image_src TEXT
    )
''')

print(card_data[8])
for card in card_data:
        cur.execute('''
        INSERT OR REPLACE INTO card_data (id, card_number, name, cost, attribute, counter, color, feature, effect, card_set, set_code, image_src)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (
        card['id'],
        card['card_number'],
        card['name'],
        card['cost'],
        card['attribute'],
        card['counter'],
        card['color'],
        card['feature'],
        card['effect'],
        card['card_set'],
        card['set_code'],
        card['image_src']
    ))
# Commit the transaction
conn.commit()

# Close the connection
conn.close()

{'id': 'ST03-005_p3', 'card_number': '005', 'name': 'Dracule Mihawk', 'cost': '4', 'attribute': 'Slash', 'counter': '2000', 'color': 'Blue', 'feature': 'he Seven Warlords of the Sea', 'effect': '[DON!! x1] [When Attacking] Draw 2 cards and trash 2 cards from your hand.', 'card_set': 'Blue Donquixote Doflamingo', 'set_code': '[ST-17]', 'image_src': 'https://asia-en.onepiece-cardgame.com//images/cardlist/card/ST03-005_p3.png?240831'}


In [128]:
series_data = soup.find('select', class_='selectModal')
#print(series_data)

set_list = []
#for series in series_data:
   
options = series_data.find_all('option')
#print(options)

set_list = [option.get('value') for option in options if option.get('value')]
    

print(set_list)

['556301', '556201', '556109', '556108', '556107', '556106', '556105', '556104', '556103', '556102', '556101', '556020', '556019', '556018', '556017', '556016', '556015', '556014', '556013', '556012', '556011', '556010', '556009', '556008', '556007', '556006', '556005', '556004', '556003', '556002', '556001', '556701', '556901', '556801']


In [9]:
connection = sqlite3.connect('OPTCGDB.db')  # Update with your DB file path
cur = connection.cursor()
cur.execute('''
    SELECT *
    FROM card_data
    WHERE name = ?
    AND set_code = ?
''', ('Trafalgar Law', 'OP07'))

cards = cur.fetchall()
print(cards)
card_map = {}
for card in cards:
    base_id = card[0].split('_')[0]  # Take base ID before underscore
    if base_id not in card_map:
        card_map[base_id] = card
connection.close()
print(list(card_map.values()))

[('OP07-047', '047', 'Trafalgar Law', 'CHARACTER', '4', 'Slash', '2000', 'Blue', 'The Seven Warlords of the Sea/Heart Pirates', "[Activate: Main] You may return this Character to the owner's hand: If your opponent has 6 or more cards in their hand, your opponent places 1 card from their hand at the bottom of their deck.", '-500 Years in the Future- [OP-07]', 'OP07', 'https://asia-en.onepiece-cardgame.com//images/cardlist/card/OP07-047.png?240831')]
[('OP07-047', '047', 'Trafalgar Law', 'CHARACTER', '4', 'Slash', '2000', 'Blue', 'The Seven Warlords of the Sea/Heart Pirates', "[Activate: Main] You may return this Character to the owner's hand: If your opponent has 6 or more cards in their hand, your opponent places 1 card from their hand at the bottom of their deck.", '-500 Years in the Future- [OP-07]', 'OP07', 'https://asia-en.onepiece-cardgame.com//images/cardlist/card/OP07-047.png?240831')]
